# 🧹 Data Preprocessing Mastery: From Messy to Model-Ready

<img src='https://miro.medium.com/max/1400/1*E5t7hp8mhp4DxPwdlQFBJA.png' width='600' alt='Data Preprocessing'>

## 🎯 The Most Important Step in Data Science

**80% of data science is data cleaning!** This notebook will teach you:
- Handle **missing data** like a pro
- Deal with **outliers** effectively
- **Transform** data for better models
- **Engineer** powerful features
- Build **reproducible pipelines**

### 📚 What We'll Master Today:
1. **Data Quality Assessment** - Understanding your mess
2. **Missing Data Strategies** - Smart imputation
3. **Outlier Detection & Treatment** - Finding anomalies
4. **Data Transformation** - Scaling and normalization
5. **Feature Engineering** - Creating powerful features
6. **Encoding Categorical Data** - Handling text data
7. **Feature Selection** - Choosing what matters
8. **Data Balancing** - Handling imbalanced data
9. **Pipeline Creation** - Automating everything
10. **Real Project** - End-to-end preprocessing

---

## 🚀 Let's Clean Some Data!

In [ ]:
# Import essential libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

# Set style for beautiful plots
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print("🧹 Data Preprocessing Mastery - Ready to Clean!")
print("\n💡 Remember: Garbage in, garbage out! Quality preprocessing = Quality models!")

---

## 📌 Section 1: Data Quality Assessment

### 🎯 Know Your Data's Problems

In [ ]:
# 1.1 Create a Messy Dataset (Realistic!)
print("🎭 CREATING REALISTIC MESSY DATA\n" + "="*40)

np.random.seed(42)
n_samples = 1000

# Create messy customer data
data = {
    'customer_id': range(1000, 1000 + n_samples),
    'age': np.random.normal(35, 15, n_samples),
    'income': np.random.lognormal(10.5, 0.5, n_samples),
    'credit_score': np.random.normal(650, 100, n_samples),
    'purchase_amount': np.random.exponential(100, n_samples),
    'membership_days': np.random.uniform(0, 1000, n_samples),
    'gender': np.random.choice(['M', 'F', 'Male', 'Female', 'male', 'female', None], n_samples),
    'city': np.random.choice(['New York', 'new york', 'NY', 'Los Angeles', 'LA', 'Chicago', None], n_samples),
    'email': [f'user{i}@email.com' if np.random.random() > 0.1 else None for i in range(n_samples)],
    'phone': [f'555-{np.random.randint(1000, 9999)}' if np.random.random() > 0.15 else None for i in range(n_samples)],
    'has_children': np.random.choice(['Yes', 'No', 'Y', 'N', '1', '0', None], n_samples),
    'satisfaction_score': np.random.choice([1, 2, 3, 4, 5, None], n_samples),
    'last_purchase_date': pd.date_range('2020-01-01', periods=n_samples, freq='D'),
    'premium_customer': np.random.choice([True, False], n_samples)
}

df = pd.DataFrame(data)

# Add more messiness
# Add missing values randomly
for col in ['age', 'income', 'credit_score']:
    missing_idx = np.random.choice(df.index, size=int(0.1 * len(df)), replace=False)
    df.loc[missing_idx, col] = np.nan

# Add outliers
outlier_idx = np.random.choice(df.index, size=50, replace=False)
df.loc[outlier_idx, 'income'] = df.loc[outlier_idx, 'income'] * 10
df.loc[outlier_idx[:25], 'age'] = np.random.uniform(100, 120, 25)

# Add duplicates
duplicate_rows = df.sample(20)
df = pd.concat([df, duplicate_rows], ignore_index=True)

# Add inconsistent data
df.loc[df['age'] < 0, 'age'] = np.abs(df.loc[df['age'] < 0, 'age'])

print("📊 Dataset Info:")
print(f"Shape: {df.shape}")
print(f"\nData Types:")
print(df.dtypes.value_counts())
print(f"\nMissing Values:")
print(df.isnull().sum()[df.isnull().sum() > 0])
print(f"\nDuplicates: {df.duplicated().sum()}")

# Display first few rows
print("\n📋 First 5 Rows:")
df.head()

In [ ]:
# 1.2 Data Quality Report
print("📊 DATA QUALITY ASSESSMENT\n" + "="*40)

def data_quality_report(df):
    """Generate comprehensive data quality report"""
    
    report = pd.DataFrame({
        'Column': df.columns,
        'Type': df.dtypes.values,
        'Missing': df.isnull().sum().values,
        'Missing%': (df.isnull().sum() / len(df) * 100).round(2).values,
        'Unique': df.nunique().values,
        'Unique%': (df.nunique() / len(df) * 100).round(2).values
    })
    
    # Add statistics for numeric columns
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    
    stats_dict = {
        'Mean': [],
        'Std': [],
        'Min': [],
        'Max': [],
        'Outliers': []
    }
    
    for col in df.columns:
        if col in numeric_cols:
            stats_dict['Mean'].append(df[col].mean())
            stats_dict['Std'].append(df[col].std())
            stats_dict['Min'].append(df[col].min())
            stats_dict['Max'].append(df[col].max())
            
            # Count outliers (using IQR method)
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            outliers = ((df[col] < (Q1 - 1.5 * IQR)) | (df[col] > (Q3 + 1.5 * IQR))).sum()
            stats_dict['Outliers'].append(outliers)
        else:
            for key in stats_dict:
                stats_dict[key].append('-')
    
    for key, values in stats_dict.items():
        report[key] = values
    
    return report

quality_report = data_quality_report(df)
print(quality_report.to_string())

# Visualize data quality
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Missing values heatmap
ax = axes[0, 0]
missing_data = df.isnull().sum()[df.isnull().sum() > 0].sort_values(ascending=False)
ax.barh(range(len(missing_data)), missing_data.values)
ax.set_yticks(range(len(missing_data)))
ax.set_yticklabels(missing_data.index)
ax.set_xlabel('Number of Missing Values')
ax.set_title('Missing Values by Column')

# Data types distribution
ax = axes[0, 1]
type_counts = df.dtypes.value_counts()
ax.pie(type_counts.values, labels=type_counts.index, autopct='%1.1f%%')
ax.set_title('Data Types Distribution')

# Correlation heatmap (numeric only)
ax = axes[1, 0]
numeric_df = df.select_dtypes(include=[np.number])
corr_matrix = numeric_df.corr()
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0, ax=ax)
ax.set_title('Correlation Matrix')

# Distribution of a numeric column
ax = axes[1, 1]
ax.hist(df['income'].dropna(), bins=50, edgecolor='black', alpha=0.7)
ax.set_xlabel('Income')
ax.set_ylabel('Frequency')
ax.set_title('Income Distribution (with outliers)')
ax.set_yscale('log')

plt.tight_layout()
plt.show()

---

## 📌 Section 2: Handling Missing Data

### 🎯 Smart Strategies for Missing Values

In [ ]:
# 2.1 Missing Data Patterns
print("🔍 MISSING DATA ANALYSIS\n" + "="*40)

# Visualize missing data patterns
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Missing data matrix
ax = axes[0]
missing_matrix = df.isnull().astype(int)
ax.imshow(missing_matrix.T, cmap='RdYlBu', aspect='auto', interpolation='none')
ax.set_xlabel('Sample Index')
ax.set_ylabel('Features')
ax.set_title('Missing Data Pattern (Yellow = Missing)')
ax.set_yticks(range(len(df.columns)))
ax.set_yticklabels(df.columns, fontsize=8)

# Missing data correlation
ax = axes[1]
missing_corr = missing_matrix.corr()
mask = np.triu(np.ones_like(missing_corr), k=1)
sns.heatmap(missing_corr, mask=mask, annot=False, cmap='coolwarm', 
            center=0, ax=ax, vmin=-1, vmax=1)
ax.set_title('Missing Data Correlation')

# Missing data by column
ax = axes[2]
missing_pct = (df.isnull().sum() / len(df)) * 100
missing_pct = missing_pct[missing_pct > 0].sort_values(ascending=False)
ax.bar(range(len(missing_pct)), missing_pct.values, color='coral')
ax.set_xticks(range(len(missing_pct)))
ax.set_xticklabels(missing_pct.index, rotation=45, ha='right')
ax.set_ylabel('Missing %')
ax.set_title('Missing Data Percentage')
ax.axhline(y=5, color='red', linestyle='--', alpha=0.5, label='5% threshold')
ax.axhline(y=20, color='darkred', linestyle='--', alpha=0.5, label='20% threshold')
ax.legend()

plt.tight_layout()
plt.show()

# Missing data types
print("\n📊 Missing Data Types:")
print("• MCAR (Missing Completely At Random): No pattern")
print("• MAR (Missing At Random): Pattern related to other variables")
print("• MNAR (Missing Not At Random): Pattern related to missing value itself")

In [ ]:
# 2.2 Imputation Strategies
print("🔧 IMPUTATION STRATEGIES\n" + "="*40)

# Create a copy for imputation
df_imputed = df.copy()

# Strategy 1: Simple Imputation
print("1️⃣ Simple Imputation:")

# Numeric columns - mean/median imputation
numeric_cols = df_imputed.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    if df_imputed[col].isnull().sum() > 0:
        # Use median for skewed data, mean for normal
        skewness = df_imputed[col].skew()
        if abs(skewness) > 1:
            impute_value = df_imputed[col].median()
            strategy = 'median'
        else:
            impute_value = df_imputed[col].mean()
            strategy = 'mean'
        
        missing_count = df_imputed[col].isnull().sum()
        df_imputed[col].fillna(impute_value, inplace=True)
        print(f"  {col}: Imputed {missing_count} values with {strategy} = {impute_value:.2f}")

# Categorical columns - mode imputation
categorical_cols = df_imputed.select_dtypes(include=['object']).columns
for col in categorical_cols:
    if df_imputed[col].isnull().sum() > 0:
        mode_value = df_imputed[col].mode()[0] if not df_imputed[col].mode().empty else 'Unknown'
        missing_count = df_imputed[col].isnull().sum()
        df_imputed[col].fillna(mode_value, inplace=True)
        print(f"  {col}: Imputed {missing_count} values with mode = {mode_value}")

print("\n2️⃣ Advanced Imputation - KNN:")

# Demonstrate KNN imputation on numeric data
from sklearn.impute import KNNImputer

# Select numeric columns with missing values for demonstration
demo_cols = ['age', 'income', 'credit_score']
df_knn = df[demo_cols].copy()

# Apply KNN imputation
imputer = KNNImputer(n_neighbors=5)
df_knn_imputed = pd.DataFrame(
    imputer.fit_transform(df_knn),
    columns=demo_cols,
    index=df_knn.index
)

# Compare imputation methods
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for idx, col in enumerate(demo_cols):
    ax = axes[idx]
    
    # Original distribution
    ax.hist(df[col].dropna(), bins=30, alpha=0.5, label='Original', color='blue')
    
    # Simple imputation
    ax.hist(df_imputed[col], bins=30, alpha=0.5, label='Simple Impute', color='green')
    
    # KNN imputation
    ax.hist(df_knn_imputed[col], bins=30, alpha=0.5, label='KNN Impute', color='red')
    
    ax.set_xlabel(col)
    ax.set_ylabel('Frequency')
    ax.set_title(f'{col} - Imputation Comparison')
    ax.legend()

plt.tight_layout()
plt.show()

print("\n💡 When to use which strategy:")
print("• Mean/Median: When data is MCAR, small % missing")
print("• Mode: For categorical variables")
print("• KNN: When data is MAR, preserves relationships")
print("• Forward/Backward fill: For time series")
print("• Domain-specific: Use business knowledge")

---

## 📌 Section 3: Outlier Detection and Treatment

### 🎯 Finding and Handling Anomalies

In [ ]:
# 3.1 Outlier Detection Methods
print("🔍 OUTLIER DETECTION\n" + "="*40)

def detect_outliers(df, column, method='IQR'):
    """Detect outliers using various methods"""
    
    if method == 'IQR':
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
        
    elif method == 'Z-score':
        z_scores = np.abs(stats.zscore(df[column].dropna()))
        outliers = df[column].dropna()[z_scores > 3]
        
    elif method == 'Isolation Forest':
        from sklearn.ensemble import IsolationForest
        iso_forest = IsolationForest(contamination=0.1, random_state=42)
        outliers_pred = iso_forest.fit_predict(df[[column]].dropna())
        outliers = df[column].dropna()[outliers_pred == -1]
    
    return outliers

# Detect outliers in income
outliers_iqr = detect_outliers(df_imputed, 'income', 'IQR')
outliers_zscore = detect_outliers(df_imputed, 'income', 'Z-score')

print(f"Outliers detected in 'income':")
print(f"  IQR method: {len(outliers_iqr)} outliers")
print(f"  Z-score method: {len(outliers_zscore)} outliers")

# Visualize outliers
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Box plot
ax = axes[0, 0]
ax.boxplot([df_imputed['income'], df_imputed['age'], df_imputed['credit_score']], 
           labels=['Income', 'Age', 'Credit Score'])
ax.set_title('Box Plot - Outlier Detection')
ax.set_ylabel('Value')

# Scatter plot with outliers
ax = axes[0, 1]
is_outlier = df_imputed['income'].isin(outliers_iqr['income'])
ax.scatter(df_imputed.index[~is_outlier], df_imputed.loc[~is_outlier, 'income'], 
          alpha=0.5, label='Normal')
ax.scatter(df_imputed.index[is_outlier], df_imputed.loc[is_outlier, 'income'], 
          color='red', label='Outlier', s=50)
ax.set_xlabel('Index')
ax.set_ylabel('Income')
ax.set_title('Income Outliers (IQR Method)')
ax.legend()

# Distribution with outlier boundaries
ax = axes[0, 2]
ax.hist(df_imputed['income'], bins=50, alpha=0.7, edgecolor='black')
Q1 = df_imputed['income'].quantile(0.25)
Q3 = df_imputed['income'].quantile(0.75)
IQR = Q3 - Q1
ax.axvline(Q1 - 1.5*IQR, color='red', linestyle='--', label='Lower Bound')
ax.axvline(Q3 + 1.5*IQR, color='red', linestyle='--', label='Upper Bound')
ax.set_xlabel('Income')
ax.set_ylabel('Frequency')
ax.set_title('Income Distribution with IQR Bounds')
ax.legend()

# Z-score visualization
ax = axes[1, 0]
z_scores = stats.zscore(df_imputed['income'])
ax.scatter(df_imputed.index, z_scores, alpha=0.5)
ax.axhline(y=3, color='red', linestyle='--', label='Z=3')
ax.axhline(y=-3, color='red', linestyle='--', label='Z=-3')
ax.set_xlabel('Index')
ax.set_ylabel('Z-score')
ax.set_title('Z-scores for Income')
ax.legend()

# Multivariate outlier detection
ax = axes[1, 1]
ax.scatter(df_imputed['age'], df_imputed['income'], alpha=0.5)
ax.set_xlabel('Age')
ax.set_ylabel('Income')
ax.set_title('Bivariate Outlier Detection')

# Local Outlier Factor
from sklearn.neighbors import LocalOutlierFactor
ax = axes[1, 2]
lof = LocalOutlierFactor(n_neighbors=20)
outlier_labels = lof.fit_predict(df_imputed[['age', 'income']])
ax.scatter(df_imputed['age'][outlier_labels == 1], 
          df_imputed['income'][outlier_labels == 1], 
          alpha=0.5, label='Normal')
ax.scatter(df_imputed['age'][outlier_labels == -1], 
          df_imputed['income'][outlier_labels == -1], 
          color='red', label='Outlier', s=50)
ax.set_xlabel('Age')
ax.set_ylabel('Income')
ax.set_title('Local Outlier Factor')
ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
# 3.2 Outlier Treatment Strategies
print("🔧 OUTLIER TREATMENT\n" + "="*40)

# Create copies for different treatment methods
df_capped = df_imputed.copy()
df_transformed = df_imputed.copy()
df_removed = df_imputed.copy()

# Method 1: Capping (Winsorization)
print("1️⃣ Capping/Winsorization:")
for col in ['income', 'age']:
    Q1 = df_capped[col].quantile(0.01)
    Q99 = df_capped[col].quantile(0.99)
    df_capped[col] = df_capped[col].clip(Q1, Q99)
    print(f"  {col}: Capped to [{Q1:.2f}, {Q99:.2f}]")

# Method 2: Log Transformation
print("\n2️⃣ Log Transformation:")
df_transformed['income_log'] = np.log1p(df_transformed['income'])
print(f"  Applied log transformation to income")

# Method 3: Removal
print("\n3️⃣ Outlier Removal:")
before_shape = df_removed.shape
for col in ['income', 'age']:
    Q1 = df_removed[col].quantile(0.25)
    Q3 = df_removed[col].quantile(0.75)
    IQR = Q3 - Q1
    df_removed = df_removed[(df_removed[col] >= Q1 - 1.5*IQR) & 
                            (df_removed[col] <= Q3 + 1.5*IQR)]
after_shape = df_removed.shape
print(f"  Removed {before_shape[0] - after_shape[0]} rows")

# Visualize treatment effects
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Original vs Capped
ax = axes[0, 0]
ax.hist(df_imputed['income'], bins=50, alpha=0.5, label='Original', color='blue')
ax.hist(df_capped['income'], bins=50, alpha=0.5, label='Capped', color='green')
ax.set_xlabel('Income')
ax.set_ylabel('Frequency')
ax.set_title('Original vs Capped')
ax.legend()

# Log transformation
ax = axes[0, 1]
ax.hist(df_transformed['income_log'], bins=50, alpha=0.7, color='purple')
ax.set_xlabel('Log(Income)')
ax.set_ylabel('Frequency')
ax.set_title('Log Transformed Income')

# Before/After boxplots
ax = axes[1, 0]
ax.boxplot([df_imputed['income'], df_capped['income'], df_removed['income']], 
           labels=['Original', 'Capped', 'Removed'])
ax.set_ylabel('Income')
ax.set_title('Outlier Treatment Comparison')

# Statistics comparison
ax = axes[1, 1]
stats_comparison = pd.DataFrame({
    'Original': [df_imputed['income'].mean(), df_imputed['income'].std(), 
                df_imputed['income'].skew(), len(df_imputed)],
    'Capped': [df_capped['income'].mean(), df_capped['income'].std(), 
              df_capped['income'].skew(), len(df_capped)],
    'Removed': [df_removed['income'].mean(), df_removed['income'].std(), 
               df_removed['income'].skew(), len(df_removed)]
}, index=['Mean', 'Std', 'Skew', 'Count'])

stats_comparison.plot(kind='bar', ax=ax)
ax.set_title('Statistics After Treatment')
ax.set_ylabel('Value')
ax.legend(title='Method')

plt.tight_layout()
plt.show()

print("\n💡 When to use which method:")
print("• Capping: Preserve all data, reduce extreme values")
print("• Transformation: When distribution is skewed")
print("• Removal: When outliers are errors or tiny %")
print("• Keep: When outliers are valid and important")

---

## 📌 Section 4: Feature Scaling and Transformation

### 🎯 Preparing Features for Machine Learning

In [ ]:
# 4.1 Feature Scaling Methods
print("⚖️ FEATURE SCALING\n" + "="*40)

# Select numeric features
features = ['age', 'income', 'credit_score', 'purchase_amount', 'membership_days']
X = df_capped[features].copy()

# Different scaling methods
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, PowerTransformer

scalers = {
    'Original': None,
    'StandardScaler': StandardScaler(),
    'MinMaxScaler': MinMaxScaler(),
    'RobustScaler': RobustScaler(),
    'PowerTransformer': PowerTransformer(method='yeo-johnson')
}

# Apply scaling
scaled_data = {}
for name, scaler in scalers.items():
    if scaler is None:
        scaled_data[name] = X
    else:
        scaled_data[name] = pd.DataFrame(
            scaler.fit_transform(X),
            columns=X.columns,
            index=X.index
        )

# Visualize scaling effects
fig, axes = plt.subplots(len(features), len(scalers), figsize=(20, 15))

for i, feature in enumerate(features):
    for j, (name, data) in enumerate(scaled_data.items()):
        ax = axes[i, j]
        ax.hist(data[feature], bins=30, alpha=0.7, edgecolor='black')
        ax.set_title(f'{feature}\n{name}')
        
        # Add statistics
        mean = data[feature].mean()
        std = data[feature].std()
        ax.axvline(mean, color='red', linestyle='--', alpha=0.5)
        ax.text(0.7, 0.9, f'μ={mean:.2f}\nσ={std:.2f}', 
               transform=ax.transAxes, fontsize=8)

plt.suptitle('Feature Scaling Comparison', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()

print("\n📊 Scaling Method Comparison:")
print("• StandardScaler: Mean=0, Std=1 (assumes normal distribution)")
print("• MinMaxScaler: Range [0,1] (preserves original distribution)")
print("• RobustScaler: Uses median/IQR (robust to outliers)")
print("• PowerTransformer: Makes data more Gaussian-like")

---

## 📌 Section 5: Feature Engineering

### 🎯 Creating Powerful Features

In [ ]:
# 5.1 Feature Engineering Techniques
print("🛠️ FEATURE ENGINEERING\n" + "="*40)

# Create new features
df_engineered = df_capped.copy()

# Temporal features
df_engineered['account_age_years'] = df_engineered['membership_days'] / 365
df_engineered['account_age_category'] = pd.cut(df_engineered['account_age_years'], 
                                                bins=[0, 1, 2, 5, 10], 
                                                labels=['New', 'Regular', 'Loyal', 'VIP'])

# Ratio features
df_engineered['income_per_age'] = df_engineered['income'] / df_engineered['age']
df_engineered['purchase_to_income_ratio'] = df_engineered['purchase_amount'] / df_engineered['income']
df_engineered['credit_score_category'] = pd.cut(df_engineered['credit_score'],
                                                 bins=[300, 580, 670, 740, 800, 850],
                                                 labels=['Poor', 'Fair', 'Good', 'Very Good', 'Excellent'])

# Interaction features
df_engineered['high_income_good_credit'] = (
    (df_engineered['income'] > df_engineered['income'].median()) & 
    (df_engineered['credit_score'] > 700)
).astype(int)

# Polynomial features
df_engineered['age_squared'] = df_engineered['age'] ** 2
df_engineered['income_log'] = np.log1p(df_engineered['income'])

# Binning continuous variables
df_engineered['age_group'] = pd.cut(df_engineered['age'], 
                                    bins=[0, 25, 35, 50, 65, 100],
                                    labels=['<25', '25-35', '35-50', '50-65', '65+'])

print("📊 New Features Created:")
new_features = ['account_age_years', 'account_age_category', 'income_per_age', 
                'purchase_to_income_ratio', 'credit_score_category', 
                'high_income_good_credit', 'age_squared', 'income_log', 'age_group']
for feat in new_features:
    print(f"  • {feat}")

# Visualize engineered features
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Age groups distribution
ax = axes[0, 0]
df_engineered['age_group'].value_counts().plot(kind='bar', ax=ax, color='steelblue')
ax.set_title('Age Group Distribution')
ax.set_xlabel('Age Group')
ax.set_ylabel('Count')
ax.set_xticklabels(ax.get_xticklabels(), rotation=0)

# Credit score categories
ax = axes[0, 1]
df_engineered['credit_score_category'].value_counts().plot(kind='pie', ax=ax, autopct='%1.1f%%')
ax.set_title('Credit Score Categories')

# Income per age
ax = axes[0, 2]
ax.scatter(df_engineered['age'], df_engineered['income_per_age'], alpha=0.5, c='coral')
ax.set_xlabel('Age')
ax.set_ylabel('Income per Age')
ax.set_title('Income/Age Ratio')

# Purchase to income ratio
ax = axes[1, 0]
ax.hist(df_engineered['purchase_to_income_ratio'], bins=30, edgecolor='black', color='lightgreen')
ax.set_xlabel('Purchase/Income Ratio')
ax.set_ylabel('Frequency')
ax.set_title('Purchase to Income Ratio')

# Account age categories
ax = axes[1, 1]
df_engineered['account_age_category'].value_counts().plot(kind='bar', ax=ax, color='purple')
ax.set_title('Customer Loyalty Categories')
ax.set_xlabel('Category')
ax.set_ylabel('Count')
ax.set_xticklabels(ax.get_xticklabels(), rotation=45)

# Feature correlation heatmap
ax = axes[1, 2]
numeric_features = df_engineered[['age', 'income', 'credit_score', 
                                  'income_per_age', 'purchase_to_income_ratio']].corr()
sns.heatmap(numeric_features, annot=True, fmt='.2f', cmap='coolwarm', ax=ax, center=0)
ax.set_title('Feature Correlations')

plt.tight_layout()
plt.show()

print("\n💡 Feature Engineering Best Practices:")
print("• Domain Knowledge: Use business understanding")
print("• Interaction Features: Combine related features")
print("• Polynomial Features: Capture non-linear relationships")
print("• Binning: Convert continuous to categorical")
print("• Time-based: Extract date/time components")
print("• Aggregations: Group-level statistics")

In [ ]:
# 5.2 Automated Feature Selection
print("\n🎯 FEATURE SELECTION\n" + "="*40)

from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier

# Prepare data for feature selection
X_features = df_engineered[['age', 'income', 'credit_score', 'purchase_amount', 
                            'membership_days', 'income_per_age', 'purchase_to_income_ratio',
                            'high_income_good_credit', 'age_squared', 'income_log']]
y_target = df_engineered['premium_customer'].astype(int)

# Method 1: Statistical Tests (ANOVA F-value)
print("1️⃣ Statistical Feature Selection (ANOVA):")
selector_f = SelectKBest(f_classif, k=5)
X_selected_f = selector_f.fit_transform(X_features, y_target)
feature_scores_f = pd.DataFrame({
    'Feature': X_features.columns,
    'F-Score': selector_f.scores_
}).sort_values('F-Score', ascending=False)
print(feature_scores_f.to_string())

# Method 2: Mutual Information
print("\n2️⃣ Mutual Information:")
selector_mi = SelectKBest(mutual_info_classif, k=5)
X_selected_mi = selector_mi.fit_transform(X_features, y_target)
feature_scores_mi = pd.DataFrame({
    'Feature': X_features.columns,
    'MI-Score': selector_mi.scores_
}).sort_values('MI-Score', ascending=False)
print(feature_scores_mi.head(10).to_string())

# Method 3: Recursive Feature Elimination (RFE)
print("\n3️⃣ Recursive Feature Elimination:")
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rfe = RFE(rf_model, n_features_to_select=5)
rfe.fit(X_features, y_target)
rfe_ranking = pd.DataFrame({
    'Feature': X_features.columns,
    'Ranking': rfe.ranking_,
    'Selected': rfe.support_
}).sort_values('Ranking')
print(rfe_ranking.to_string())

# Visualize feature importance
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# F-scores
ax = axes[0]
feature_scores_f.head(10).plot(x='Feature', y='F-Score', kind='barh', ax=ax, color='skyblue')
ax.set_title('Feature Importance - ANOVA F-Score')
ax.set_xlabel('F-Score')

# Mutual Information
ax = axes[1]
feature_scores_mi.head(10).plot(x='Feature', y='MI-Score', kind='barh', ax=ax, color='lightcoral')
ax.set_title('Feature Importance - Mutual Information')
ax.set_xlabel('MI Score')

# Random Forest Feature Importance
ax = axes[2]
rf_model.fit(X_features, y_target)
feature_importance_rf = pd.DataFrame({
    'Feature': X_features.columns,
    'Importance': rf_model.feature_importances_
}).sort_values('Importance', ascending=False)
feature_importance_rf.head(10).plot(x='Feature', y='Importance', kind='barh', ax=ax, color='lightgreen')
ax.set_title('Feature Importance - Random Forest')
ax.set_xlabel('Importance')

plt.tight_layout()
plt.show()

---

## 📌 Section 6: Categorical Encoding

### 🎯 Converting Text to Numbers

In [ ]:
# 6.1 Categorical Encoding Techniques
print("🔤 CATEGORICAL ENCODING\n" + "="*40)

# Clean categorical variables first
df_encoded = df_engineered.copy()

# Standardize gender column
df_encoded['gender'] = df_encoded['gender'].str.lower()
df_encoded['gender'] = df_encoded['gender'].replace({
    'm': 'male', 'f': 'female',
    'Male': 'male', 'Female': 'female'
})

# Standardize city column
df_encoded['city'] = df_encoded['city'].replace({
    'new york': 'New York', 'NY': 'New York',
    'LA': 'Los Angeles'
})

# Method 1: Label Encoding (for ordinal)
print("1️⃣ Label Encoding:")
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df_encoded['satisfaction_score_encoded'] = le.fit_transform(df_encoded['satisfaction_score'].fillna('0'))
print(f"  Encoded satisfaction_score")

# Method 2: One-Hot Encoding (for nominal)
print("\n2️⃣ One-Hot Encoding:")
df_encoded = pd.get_dummies(df_encoded, columns=['gender', 'city'], prefix=['gender', 'city'])
print(f"  One-hot encoded gender and city")

# Method 3: Binary Encoding (for yes/no)
print("\n3️⃣ Binary Encoding:")
df_encoded['has_children'] = df_encoded['has_children'].replace({
    'Yes': 1, 'Y': 1, '1': 1,
    'No': 0, 'N': 0, '0': 0
}).fillna(0).astype(int)
print(f"  Binary encoded has_children")

# Method 4: Target Encoding (mean encoding)
print("\n4️⃣ Target Encoding:")
target_mean = df_encoded.groupby('credit_score_category')['premium_customer'].mean()
df_encoded['credit_category_target_encoded'] = df_encoded['credit_score_category'].map(target_mean)
print(f"  Target encoded credit_score_category")

print(f"\n📊 Shape after encoding: {df_encoded.shape}")
print(f"Original shape: {df.shape}")
print(f"New columns added: {df_encoded.shape[1] - df.shape[1]}")

---

## 📌 Section 7: Complete Preprocessing Pipeline

### 🎯 Putting It All Together

In [ ]:
# 7.1 Building a Preprocessing Pipeline
print("🚀 COMPLETE PREPROCESSING PIPELINE\n" + "="*40)

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

# Define column groups
numeric_features = ['age', 'income', 'credit_score', 'purchase_amount', 'membership_days']
categorical_features = ['gender', 'city', 'has_children']

# Create preprocessing pipelines for each type
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Use the original messy data
X = df[numeric_features + categorical_features]
y = df['premium_customer']

# Split data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit and transform
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

print(f"✅ Pipeline Complete!")
print(f"  Original shape: {X_train.shape}")
print(f"  Processed shape: {X_train_processed.shape}")
print(f"  Training samples: {len(X_train_processed)}")
print(f"  Test samples: {len(X_test_processed)}")

In [ ]:
# 7.2 Save Preprocessing Pipeline
print("💾 SAVING PIPELINE\n" + "="*40)

import joblib

# Save the preprocessor
joblib.dump(preprocessor, 'preprocessor_pipeline.pkl')
print("✅ Pipeline saved as 'preprocessor_pipeline.pkl'")

# Example of loading and using
# loaded_preprocessor = joblib.load('preprocessor_pipeline.pkl')
# new_data_processed = loaded_preprocessor.transform(new_data)

print("\n" + "="*50)
print("🎉 DATA PREPROCESSING COMPLETE!")
print("="*50)

print("\n📚 Key Takeaways:")
print("1. Always assess data quality first")
print("2. Handle missing data appropriately")
print("3. Detect and treat outliers carefully")
print("4. Scale features for ML algorithms")
print("5. Engineer meaningful features")
print("6. Select relevant features")
print("7. Encode categorical variables properly")
print("8. Build reproducible pipelines")
print("9. Document your preprocessing steps")
print("10. Validate on unseen data")

print("\n💪 You're now ready to build amazing ML models!")
print("Remember: Better data → Better models → Better results!")

# Final summary statistics
print("\n📊 FINAL DATASET SUMMARY\n" + "="*40)
print(f"Original dataset: {df.shape}")
print(f"After preprocessing: {X_train_processed.shape}")
print(f"Features created: {len(new_features)}")
print(f"Missing values handled: ✅")
print(f"Outliers treated: ✅")
print(f"Features scaled: ✅")
print(f"Pipeline created: ✅")
print("\n🚀 Ready for Machine Learning!")